In [1]:
from satellite_czml import satellite_czml
import urllib

In [2]:

url = 'https://celestrak.com/NORAD/elements/stations.txt'
tles = [l.decode("utf-8").strip() for l in urllib.request.urlopen(url).readlines()]
tle_list = [[tles[i],tles[i+1],tles[i+2]] for i,_ in enumerate(tles) if i%3==0]
tle_list

[['ISS (ZARYA)',
  '1 25544U 98067A   21014.87156498  .00001899  00000-0  42163-4 0  9993',
  '2 25544  51.6460  21.0493 0000363 230.6069 201.6701 15.49294771264830'],
 ['KESTREL EYE IIM (KE2M)',
  '1 42982U 98067NE  21014.77805526  .00023808  00000-0  13091-3 0  9991',
  '2 42982  51.6337 276.6097 0003876 168.7758 191.3330 15.79604657184356'],
 ['DELLINGR (RBLE)',
  '1 43021U 98067NJ  21014.75865909  .00032453  00000-0  16062-3 0  9992',
  '2 43021  51.6310 273.5291 0003555 136.3894 223.7389 15.81721620180162'],
 ['TEMPEST-D',
  '1 43547U 98067NV  21014.80161553  .00010623  00000-0  85389-4 0  9999',
  '2 43547  51.6355 320.5451 0006310 120.5001 239.6620 15.71593906142628'],
 ['RADSAT-G',
  '1 43553U 98067PB  21014.60066221  .00056317  00000-0  23537-3 0  9996',
  '2 43553  51.6368 291.5038 0006009 151.5764 208.5568 15.85040856143423'],
 ['AEROCUBE 12A',
  '1 43556U 18046C   21014.71067350  .00002275  00000-0  87340-4 0  9991',
  '2 43556  51.6391 169.8828 0007455 335.2106  24.8516 15

## Simple Example

In [3]:
iss_tle = tle_list[0]
iss_tle

['ISS (ZARYA)',
 '1 25544U 98067A   21014.87156498  .00001899  00000-0  42163-4 0  9993',
 '2 25544  51.6460  21.0493 0000363 230.6069 201.6701 15.49294771264830']

In [4]:
iss_czml = satellite_czml([iss_tle]).get_czml()

## Multiple Satellite Example

In [5]:
all_czml = satellite_czml(tle_list).get_czml()

## Specifying Arguments

In [6]:
from datetime import datetime, timedelta
import random

start_time = datetime.now() - timedelta(days=1)
end_time = datetime.now()
name_list = [t[0] for t in tle_list]
description_list = ['Station: ' + t[0] for t in tle_list]
color_list = [[random.randrange(256) for x in range(3)] for x in range(len(tle_list))]
size_list = [7] * len(tle_list)

czml_obj = satellite_czml(tle_list=tle_list, name_list=name_list, description_list=description_list,
                          color_list=color_list, speed_multiplier=1, use_default_image=False,
                          marker_scale_list=size_list, show_label=False, show_path=False,
                          ignore_bad_tles=True)
all_czml_p = czml_obj.get_czml()

TypeError: __init__() got an unexpected keyword argument 'show_label'

## Create One-by-One

## Modifying an Object